In [28]:
import glob
import os
import pandas as pd
from sklearn import model_selection

In [2]:
train=[i for i in glob.glob('D:\\Dataset\\DOGvsCAT\\train\\*')]
targets=[ 0 if files.split('\\')[-1][0:3]=='cat' else 1 for files in glob.glob('D:\\Dataset\\DOGvsCAT\\train\\*')]
dict_={'images': train, 'targets': targets}
df=pd.DataFrame(dict_)
df.head()

,images,targets
0,D:\Dataset\DOGvsCAT\train\cat.0.jpg,0
1,D:\Dataset\DOGvsCAT\train\cat.1.jpg,0
2,D:\Dataset\DOGvsCAT\train\cat.10.jpg,0
3,D:\Dataset\DOGvsCAT\train\cat.100.jpg,0
4,D:\Dataset\DOGvsCAT\train\cat.1000.jpg,0


In [29]:
test_images=[i for i in glob.glob('D:\\Dataset\\DOGvsCAT\\test\\*')]


In [30]:
dict1_={'images': test_images, 'targets': np.ones(len(test_images))}
df1=pd.DataFrame(dict1_)
df1.head()

,images,targets
0,D:\Dataset\DOGvsCAT\test1\1.jpg,1.0
1,D:\Dataset\DOGvsCAT\test1\10.jpg,1.0
2,D:\Dataset\DOGvsCAT\test1\100.jpg,1.0
3,D:\Dataset\DOGvsCAT\test1\1000.jpg,1.0
4,D:\Dataset\DOGvsCAT\test1\10000.jpg,1.0


In [3]:
import os
import pandas as pd
from sklearn import model_selection

if __name__ == "__main__":
    input_path='D:\\Dataset\\DOGvsCAT\\'
    df["kfold"] = -1
    df = df.sample(frac=1).reset_index(drop=True)
    y = df.targets.values
    skf = model_selection.StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
    for fold,(idxT,idxV) in enumerate(skf.split(X=df, y=y)):
        df.loc[idxV, "kfold"] = fold
    print(df.targets.value_counts())
    df.to_csv(os.path.join(input_path, "train_folds.csv"), index=False)

1    12500
0    12500
Name: targets, dtype: int64


In [12]:
from PIL import Image
import torch
import numpy as np

class ClassificationDataset:
    
    def __init__(self, image_paths, targets, resize, augmentations=None):
        self.image_paths = image_paths
        self.targets = targets
        self.resize = resize
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, item):
        image = Image.open(self.image_paths[item])
        targets = self.targets[item]
        if self.resize is not None:
            image = image.resize(
                (self.resize[1], self.resize[0]), resample=Image.BILINEAR
            )
        image = np.array(image)
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        return {
            "image": torch.tensor(image),
            "targets": torch.tensor(targets, dtype=torch.long),
        }

In [18]:
import os
import torch
import pandas as pd
from sklearn import model_selection
import albumentations 
import pretrainedmodels
import albumentations as A
import albumentations as aug
import numpy as np
import pandas as pd
import torch.nn as nn
from sklearn.model_selection import KFold
import DogDataset
import torchvision

from apex import amp
from sklearn import metrics
from torch.nn import functional as F

from wtfml.data_loaders.image import ClassificationLoader
from wtfml.engine import Engine
from wtfml.utils import EarlyStopping
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose
)

import efficientnet_pytorch

"""
class SEResNext50_32x4d(nn.Module):
    def __init__(self, pretrained="imagenet"):
        super(SEResNext50_32x4d, self).__init__()
        self.model = pretrainedmodels.__dict__[
            "se_resnext50_32x4d"
        ](pretrained=pretrained)
        #for param in self.model.parameters():
        #    param.requires_grad=False
        self.out = nn.Linear(2048, 1)
    
    def forward(self, image, targets):
        bs, _, _, _ = image.shape
        x = self.model.features(image)
        x = F.adaptive_avg_pool2d(x, 1)
        x = x.reshape(bs, -1)
        out = self.out(x)
        loss = nn.BCEWithLogitsLoss()(
            out, targets.reshape(-1, 1).type_as(out)
        )
        return out, loss
      
class EfficientNet(nn.Module):
    def __init__(self):
        super(EfficientNet, self).__init__()
        self.base_model = efficientnet_pytorch.EfficientNet.from_pretrained(
            'efficientnet-b5'
        )
        self.base_model._fc = nn.Linear(
            in_features=2048, 
            out_features=1000, 
            bias=True
        )
        
        self.=nn.Linear(self.num_ftrs,1000)
        self.csv = nn.Sequential(nn.Linear(9, 500),
                                 nn.BatchNorm1d(500),
                                 nn.ReLU(),
                                 nn.Dropout(p=0.3),
                                 
                                 nn.Linear(500, 500),
                                 nn.BatchNorm1d(500),
                                 nn.ReLU(),
                                 nn.Dropout(p=0.3))
        #print(self.base_model)
        
    def forward(self, image, targets):
        out = self.base_model(image)
        loss = nn.BCEWithLogitsLoss()(out, targets.view(-1, 1).type_as(out))
        return out, loss
"""

class PlantModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.backbone = torchvision.models.resnet18(pretrained=True)
        
        in_features = self.backbone.fc.in_features

        self.out = nn.Linear(in_features, 2)
        
    def forward(self, image, targets=None):
        batch_size, C, H, W = image.shape
        
        x = self.backbone.conv1(image)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)
        
        x = F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)
        #x = F.dropout(x, 0.25, self.training)

        x = self.out(x)
        
        loss=None
        
        if targets is not None:
            loss = nn.CrossEntropyLoss()(x, targets)
            
        return x, loss

def train(fold):
    training_data_path = "D:\\Dataset\\DOGvsCAT\\"
    model_path = "D:\\Dataset\\DOGvsCAT\\"
    df = pd.read_csv("D:\\Dataset\\DOGvsCAT\\train_folds.csv")
    device = "cuda"
    epochs = 50
    train_bs = 16
    valid_bs = 16
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    p=0.5
    df_train=df.loc[df.kfold!=0].reset_index(drop=True)
    df_valid=df.loc[df.kfold==0].reset_index(drop=True)
    train_aug = albumentations.Compose(
        [
        A.Cutout(p=p),
        A.RandomRotate90(p=p),
        A.Flip(p=p),
        A.OneOf([
            A.RandomBrightnessContrast(brightness_limit=0.2,
                                       contrast_limit=0.2,
                                       ),
            A.HueSaturationValue(
                hue_shift_limit=20,
                sat_shift_limit=50,
                val_shift_limit=50)
        ], p=p),
        A.OneOf([
            A.IAAAdditiveGaussianNoise(),
            A.GaussNoise(),
        ], p=p),
        A.OneOf([
            A.MotionBlur(p=0.2),
            A.MedianBlur(blur_limit=3, p=0.1),
            A.Blur(blur_limit=3, p=0.1),
        ], p=p),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=p),
        A.OneOf([
            A.OpticalDistortion(p=0.3),
            A.GridDistortion(p=0.1),
            A.IAAPiecewiseAffine(p=0.3),
        ], p=p),
        A.Normalize(mean, std, max_pixel_value=255.0, always_apply=True),  
        ]

        
    )

    valid_aug = albumentations.Compose(
        [
            A.Normalize(mean, std, max_pixel_value=255.0, always_apply=True),
        ]
    )

    train_images = df_train.images.values.tolist()
    train_images = [i for i in train_images]
    train_targets = df_train.targets.values

    valid_images = df_valid.images.values.tolist()
    valid_images = [i for i in valid_images]
    valid_targets = df_valid.targets.values
    

    train_dataset = DogDataset.ClassificationDataset(
        image_paths=train_images,
        targets=train_targets,
        resize=(256,256),
        augmentations=train_aug
    )
    

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=train_bs,
        shuffle=True,
        num_workers=4
    )
    


    valid_dataset = DogDataset.ClassificationDataset(
        image_paths=valid_images,
        targets=valid_targets,
        resize=(256,256),
        augmentations=valid_aug
    )

    valid_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=valid_bs,
        shuffle=False,
        num_workers=4)

    model = PlantModel()
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        patience=3,
        mode="max"
    )

    model, optimizer = amp.initialize(
        model,
        optimizer,
        opt_level="O1",
        verbosity=0
    )

    es = EarlyStopping(patience=5, mode="max")
    for epoch in range(epochs):
        training_loss = Engine.train(
            train_loader, 
            model,
            optimizer,
            device,
            fp16=True
        )
        predictions, valid_loss = Engine.evaluate(
            valid_loader, 
            model,
            device
        )
        predictions = np.concatenate(predictions,axis=0)
        #print(predictions)
        #print(predictions.shape)
        predictions = np.argmax(predictions, axis=1)
        #print(predictions)
        #print(predictions.shape)
        auc = metrics.accuracy_score(valid_targets, predictions)
        scheduler.step(auc)
        print(f"epoch={epoch}, auc={auc}")
        es(auc, model, os.path.join(model_path, f"model{fold}.bin"))
        if es.early_stop:
            print("early stopping")
            break


def predict(fold):
    test_data_path = "D:\\Dataset\\DOGvsCAT\\"
    model_path = "D:\\Dataset\\DOGvsCAT\\"
    #df_test = pd.read_csv("D:\\Dataset\\melanoma-384-384\\test.csv")
    #df_test.loc[:, "target"] = 0
    p=0.5
    device = "cuda"
    epochs = 50
    test_bs = 16
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)

    train_aug = albumentations.Compose(
        [
            
           A.RandomRotate90(p=p),
        A.Flip(p=p),
        A.OneOf([
            A.RandomBrightnessContrast(brightness_limit=0.2,
                                       contrast_limit=0.2,
                                       ),
            A.HueSaturationValue(
                hue_shift_limit=20,
                sat_shift_limit=50,
                val_shift_limit=50)
        ], p=p),
        A.OneOf([
            A.IAAAdditiveGaussianNoise(),
            A.GaussNoise(),
        ], p=p),
        aug.Normalize(mean, std, max_pixel_value=255.0, always_apply=True),  
        ]
    )

    valid_aug = albumentations.Compose(
        [
            albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True),
        ]
    )


    test_images = df1.images.values.tolist()
    test_images = [i for i in test_images]
    test_targets = df1.targets.values

    
    test_dataset = DogDataset.ClassificationDataset(
        image_paths=test_images,
        targets=test_targets,
        resize=(256,256),
        augmentations=train_aug
    )
    #print(len(test_dataset))
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=test_bs,
        shuffle=False,
        num_workers=4
    )

    model = PlantModel()
    model.load_state_dict(torch.load(os.path.join(model_path, f"model{fold}.bin")))
    model.to(device)

    predictions = Engine.predict(
        test_loader,
        model,
        device
    )
    return np.concatenate(predictions,axis=0)


if __name__ == "__main__":
    pred1=predict(fold=0)
    

NameError: name 'df1' is not defined

In [34]:
final_sub=(pred1+pred2+pred3+pred4+pred5)/5

In [99]:
torch.set_printoptions(precision=10)
predicted_out=np.argmax(final_sub,1)
pd.options.display.float_format='{:,.4f}'.format
final_df=pd.DataFrame(torch.softmax(torch.tensor(final_sub),dim=1).cpu().numpy(),columns=['CAT','DOG',])

In [101]:
final_df['PREDICTED']=predicted_out

In [103]:
final_df.to_csv('D://Dataset//DOGvsCAT//out.csv',index=False)

In [104]:
submission=pd.read_csv('D://Dataset//DOGvsCAT//sampleSubmission.csv')

In [106]:
submission['label']=predicted_out

In [108]:
submission.to_csv('D://Dataset//DOGvsCAT//final_submission.csv',index=False)

In [17]:
def predict(
    img_path,
    fold, 
    Model
  ):
    model_path = "D:\\Dataset\\DOGvsCAT\\"
    p=0.5
    device = "cuda"
    epochs = 50
    test_bs = 16
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)


    test_aug = albumentations.Compose(
        [
            albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True),
        ]
    )


    df1=pd.DataFrame()
    df1['images']=[image_paths]
    test_images = df1.images.values.tolist()
    test_images = [i for i in test_images]
    df1['targets']=[1,]
    test_targets = df1.targets.values

    
    test_dataset = DogDataset.ClassificationDataset(
        image_paths=img_path,
        targets=test_targets,
        resize=(256,256),
        augmentations=test_aug
    )
    
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=test_bs,
        shuffle=False,
        num_workers=4
    )

    model = Model.PlantModel()
    model.load_state_dict(torch.load(os.path.join(model_path, f"model{fold}.bin")))
    model.to(device)

    predictions = Engine.predict(
        test_loader,
        model,
        device
    )
    result = np.concatenate(predictions,axis=0)
    predicted=np.argmax(result,1)
    
    return predicted

In [3]:
import pandas as pd
    df=pd.DataFrame()

In [10]:
df['target']=[1,]

In [14]:
l=['img/test/']

In [26]:
class PlantModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.backbone = torchvision.models.resnet50(pretrained=True)
        
        in_features = self.backbone.fc.in_features

        self.out = nn.Linear(in_features, 2)
        
    def forward(self, image, targets=None):
        batch_size, C, H, W = image.shape
        
        x = self.backbone.conv1(image)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)
        
        x = F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)
        #x = F.dropout(x, 0.25, self.training)

        x = self.out(x)
        
        loss=None
        
        if targets is not None:
            loss = nn.CrossEntropyLoss()(x, targets)
            
        return x, loss

In [27]:
print(PlantModel())

PlantModel(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      